In [1]:
import sys, os
import numpy as np
import pandas as pd
from scipy.stats import norm, ttest_ind, combine_pvalues
from statsmodels.stats.multitest import multipletests
import re

In [2]:
XL_data_df = pd.read_csv('EXP/ispE_U24R1_XL_0711.csv')
XL_data = {}
native_abundance_list = XL_data_df[['Abundances (Light): 1', 'Abundances (Light): 2', 'Abundances (Light): 3', 'Abundances (Light): 4', 'Abundances (Light): 5']].to_numpy()
refolded_abundance_list = XL_data_df[['Abundances (Heavy): 1', 'Abundances (Heavy): 2', 'Abundances (Heavy): 3', 'Abundances (Heavy): 4', 'Abundances (Heavy): 5']].to_numpy()

In [3]:
key_list = []
data_list = []
for idx, pair in enumerate(XL_data_df[['RealLinkage1','RealLinkage2']].to_numpy()):
    if not np.any(pd.isnull(pair)):
        resid_list = sorted(pair, key=lambda x: int(x[1:]))
        pair_str = '-'.join(resid_list)
        na_list = native_abundance_list[idx,:]
        ra_list = refolded_abundance_list[idx,:]
        
        # compute ratio and p-value
        nn = len(np.where(na_list == 1000)[0])
        nr = len(np.where(ra_list == 1000)[0])
        if nn == 0 and nr == 0: # all values available
            x1 = ra_list
            x2 = na_list
            alt_hyp = 'two-sided'
        elif (nn == 1 and nr == 0) or (nn == 0 and nr == 1): # one value is missing
            x1 = ra_list[ra_list!=1000]
            x2 = na_list[na_list!=1000]
            alt_hyp = 'two-sided'
        elif nn == len(na_list) and nr == 0: # All-or-nothing case
            x1 = ra_list
            x2 = norm.rvs(1e4, 1e3, 5)
            alt_hyp = 'greater'
        elif nn == 0 and nr == len(ra_list): # All-or-nothing case
            x1 = norm.rvs(1e4, 1e3, 5)
            x2 = na_list
            alt_hyp = 'less'
        else:
            continue
        ratio = np.mean(x1)/np.mean(x2)
        ttest_result = ttest_ind(x1, x2, alternative=alt_hyp, equal_var=False)
            
        key_list.append(pair_str)
        data_list.append([ratio, ttest_result[0], ttest_result[1]])


In [4]:
data_list = np.array(data_list)
adj_pvalue_list = multipletests(data_list[:,2], alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
data_list = np.hstack((data_list, adj_pvalue_list.reshape((len(adj_pvalue_list),1))))

In [5]:
for idx, key in enumerate(key_list):
    if key in XL_data.keys():
        XL_data[key].append(data_list[idx,:])
    else:
        XL_data[key] = [data_list[idx,:]]

In [6]:
for k, v in XL_data.items():
    XL_data[k] = np.array(v)

In [7]:
XL_data_merged = {}
for k, v in XL_data.items():
    idx_list = np.where(np.sign(v[:,1]) == np.sign(np.sign(v[:,1]).sum()))[0]
    if len(idx_list) != 0:
        ratio_merged = np.median(v[idx_list,0])
        pvalue_merged = combine_pvalues(v[idx_list,2])[1]
        adj_pvalue_merged = combine_pvalues(v[idx_list,3])[1]
    else:
        ratio_merged = np.median(v[:,0])
        pvalue_merged = 1
        adj_pvalue_merged = 1
    XL_data_merged[k] = [np.log2(ratio_merged), np.abs(-np.log10(pvalue_merged)), np.abs(-np.log10(adj_pvalue_merged))]

df = pd.DataFrame([[k,*v] for k,v in XL_data_merged.items()], columns=['Pairs', 'log2(heavy/light)', '-log10(p-value)', '-log10(Adj. p-value)'])

print(df)

        Pairs  log2(heavy/light)  -log10(p-value)  -log10(Adj. p-value)
0    K96-S208          -5.325934         3.439445              3.085078
1    K76-K196          -1.851383         5.922372              5.080254
2   K196-K271          -2.925204         5.369038              4.630058
3    K96-K196          -5.325765        12.964370             10.106890
4    K76-S208          -5.554157         2.004652              1.862775
5   K196-K204          -3.895533        17.934922             14.536706
6   K196-S208          -6.829610        11.186853              8.143803
7    K10-K196          -7.239391         2.942025              2.566900
8    Y16-K196          -2.722340         2.828670              2.490498
9   K196-S198          -1.717924         6.016972              5.065361
10  K196-S276          -1.341094         1.465556              1.379196
11  K271-S285          -2.838008         2.763328              2.440396
12  K196-T201           4.825826         0.000000              0

In [27]:
# Only select crosslinking with significant abundance change (log2(R/N) >1 and the difference is significant after FDR correction: adj. p-value <=0.05 ~ -log10(adj. pvalue) > 1.301
df_significant = df[(abs(df['log2(heavy/light)'])>=1) & (df['-log10(Adj. p-value)']>1.301)]

In [28]:
# Function to extract the first residue index
def get_first_residue_index(pair):
    match = re.match(r'^[A-Z](\d+)', pair)
    if match:
        return match.group(0), match.group(1)#, match.group(2), match.group(3)
        # return int(match.group(1))
    return float('inf')  # if no match, place at the end


# Function to extract the first residue index
def get_residue_indices(pair):
    match = re.match(r'^([A-Z])(\d+)-([A-Z])(\d+)$', pair)
    if match:
        # return match.group()
        return int(match.group(2)), int(match.group(4))
    return float('inf'), float('inf')  # if no match, place at the end


In [29]:
df_significant[['First_Residue_Index', 'Last_Residue_Index']] = df_significant['Pairs'].apply(lambda x: pd.Series(get_residue_indices(x)))

/tmp/ipykernel_4178846/1876909240.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_significant[['First_Residue_Index', 'Last_Residue_Index']] = df_significant['Pairs'].apply(lambda x: pd.Series(get_residue_indices(x)))
/tmp/ipykernel_4178846/1876909240.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_significant[['First_Residue_Index', 'Last_Residue_Index']] = df_significant['Pairs'].apply(lambda x: pd.Series(get_residue_indices(x)))


In [30]:
df_significant

,Pairs,log2(heavy/light),-log10(p-value),-log10(Adj. p-value),First_Residue_Index,Last_Residue_Index
0,K96-S208,-5.325934,3.439445,3.085078,96,208
1,K76-K196,-1.851383,5.922372,5.080254,76,196
2,K196-K271,-2.925204,5.369038,4.630058,196,271
3,K96-K196,-5.325765,12.964370,10.106890,96,196
4,K76-S208,-5.554157,2.004652,1.862775,76,208
5,K196-K204,-3.895533,17.934922,14.536706,196,204
6,K196-S208,-6.829610,11.186853,8.143803,196,208
7,K10-K196,-7.239391,2.942025,2.566900,10,196
8,Y16-K196,-2.722340,2.828670,2.490498,16,196
9,K196-S198,-1.717924,6.016972,5.065361,196,198


In [31]:
df_significant = df_significant.sort_values(by=['First_Residue_Index', 'Last_Residue_Index']).drop(columns=['First_Residue_Index', 'Last_Residue_Index']).reset_index(drop=True)

In [32]:
df_significant

,Pairs,log2(heavy/light),-log10(p-value),-log10(Adj. p-value)
0,S7-Y16,-1.535149,1.379405,1.301646
1,K10-K196,-7.239391,2.942025,2.566900
2,Y16-K196,-2.722340,2.828670,2.490498
3,K76-T86,-4.528623,12.175333,8.788980
4,K76-T161,-11.932206,2.345426,2.116398
5,K76-K196,-1.851383,5.922372,5.080254
6,K76-S208,-5.554157,2.004652,1.862775
7,T77-S208,4.742097,1.924206,1.801634
8,S81-T86,-1.204474,3.295572,2.721556
9,S81-K96,-1.883878,9.786008,7.349076


In [33]:
df_significant.to_excel('EXP/ispE_XLMS_exp_U24R1_XL_0711_processed.xlsx')